In [84]:
import csv
import pandas as pd
import numpy as np
import urllib
from tabula.io import read_pdf, convert_into
import datetime
from typing import List, Tuple, Any

from config import Config
from pgConnect import PgConnection
from dfs_dao import Dfs_dao

In [75]:
config : Config = Config()
pgc : PgConnection = PgConnection(config)
dao : Dfs_dao = Dfs_dao(pgc)

Connecting to the PostgreSQL database...


In [76]:
injury_base = 'https://ak-static.cms.nba.com/referee/injury/Injury-Report'

In [77]:
link = 'https://ak-static.cms.nba.com/referee/injury/Injury-Report_2023-01-18_03PM.pdf'

In [122]:
def process_times_of_day(game_time_1 : datetime.time, game_time_2 : datetime.time) -> List[str]:
    hr1 = game_time_1.hour - 1
    hr2 = game_time_2.hour - 1
    
    retHours = []
    ampm = 'AM'
    while hr1 < hr2 + 2:
        printHr = hr1
        if hr1 >= 12:
            ampm = 'PM'
        if hr1 > 12:
            printHr -= 12
        retHours.append((str(printHr).zfill(2) + ampm, hr1))
        hr1 += 1
    return retHours

In [123]:
date_tuples = dao.select_game_date_times()

def get_injury_report_links(data_tuples : List[Tuple[Any,... ]]) -> List[Tuple[str, datetime.time]]:
    collection_hours = []
    time_stamps = []
    for date_tuple in date_tuples:
        yr, mth, day = (str(date_tuple[0].year), 
                        str(date_tuple[0].month).zfill(2), 
                        str(date_tuple[0].day).zfill(2))
        
        day_base = '-'.join([yr, mth, day])
        hours = process_times_of_day(date_tuple[1], date_tuple[2])
        for hour in hours:
            collection_hours.append(injury_base + '_' + day_base + '_' + hour[0] + '.pdf')
            time_stamps.append(datetime.time(hour[1],30))
    return list(zip(collection_hours, time_stamps))

test = get_injury_report_links(date_tuples)

In [124]:
test = test[:5]

In [148]:
print(test[1])

('https://ak-static.cms.nba.com/referee/injury/Injury-Report_2023-01-17_07PM.pdf', datetime.time(19, 30))


In [152]:
def extract_date_from_link(link : str) -> datetime.date:
    date_portion : str = link[link.rfind('t_')+2 : link.rfind('_')]
    dp_list : List[str] = [int(i) for i in date_portion.split('-')]
    dt : datetime.date = datetime.date(*tuple(dp_list))
    return dt


In [ ]:
convert_into('https://ak-static.cms.nba.com/referee/injury/Injury-Report_2022-10-15_11AM.pdf', 'test.csv', pages = 'all')

In [181]:
def collect_injury_report_csv(link : str, report_time : datetime.time):
    dt : datetime.date = extract_date_from_link(link)

    response = urllib.request.urlopen(link)
    a = response.read()
    convert_into(link, 'test.csv', pages = 'all')a

    with open('test.csv') as f:
        csv_rows = []
        r = csv.reader(f)
        for num, row in enumerate(r):
            row = [''] * 7 + row
            while row[-1] == '':
                row.pop(-1)
            row = row[-7:]
            if row[-1] not in ('NOT YET SUBMITTED', 'Reason') or num == 0:
                csv_rows.append(row)

    df = pd.DataFrame(csv_rows[1:], columns = csv_rows[0])
    df = df.replace('', np.nan).fillna(method = 'ffill')
    df['report_date'] = dt
    df['report_time'] = report_time
    df['report_datetime'] = datetime.datetime.combine(dt, report_time)
    tmp = df['Game Time'].str.replace(' (ET)', 'pm', regex = False)
    df['game_datetime'] = pd.to_datetime(df['Game Date'] + ' ' + tmp, format = '%m/%d/%Y %I:%M%p')
    return df

a = collect_injury_report_csv(test[0][0], test[0][1])

Got stderr: Jan 18, 2023 6:37:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 18, 2023 6:37:32 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [157]:
datetime.time(int('07'), int('30'))

datetime.time(7, 30)

In [182]:
a.head()

,Game Date,Game Time,Matchup,Team,Player Name,Current Status,Reason,report_date,report_time,report_datetime,game_datetime
0,01/17/2023,07:30 (ET),TOR@MIL,Milwaukee Bucks,"Antetokounmpo, Giannis",Out,Injury/Illness - Left Knee; Soreness,2023-01-17,18:30:00,2023-01-17 18:30:00,2023-01-17 19:30:00
1,01/17/2023,07:30 (ET),TOR@MIL,Milwaukee Bucks,"Ibaka, Serge",Out,Personal Reasons,2023-01-17,18:30:00,2023-01-17 18:30:00,2023-01-17 19:30:00
2,01/17/2023,07:30 (ET),TOR@MIL,Milwaukee Bucks,"Middleton, Khris",Out,Injury/Illness - Right Knee; Soreness,2023-01-17,18:30:00,2023-01-17 18:30:00,2023-01-17 19:30:00
3,01/17/2023,07:30 (ET),TOR@MIL,Toronto Raptors,"Banton, Dalano",Out,Injury/Illness - Right Hip; Pointer,2023-01-17,18:30:00,2023-01-17 18:30:00,2023-01-17 19:30:00
4,01/17/2023,07:30 (ET),TOR@MIL,Toronto Raptors,"Porter Jr., Otto",Out,Injury/Illness - Left Foot; Surgery - second t...,2023-01-17,18:30:00,2023-01-17 18:30:00,2023-01-17 19:30:00


In [41]:
from datetime import timedelta
datetime.time(19, 0) + timedelta(hours = -1)a

TypeError: unsupported operand type(s) for +: 'datetime.time' and 'datetime.timedelta'

Got stderr: Jan 18, 2023 4:29:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 18, 2023 4:29:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Game Date,Game Time,Matchup,Team,Player Name,Current Status,Reason
0,01/18/2023,07:30 (ET),ATL@DAL,Atlanta Hawks,"Williams, Donovan",Out,G League - Two-Way
1,01/18/2023,07:30 (ET),ATL@DAL,Dallas Mavericks,"Doncic, Luka",Probable,Injury/Illness - Left Ankle; Soreness
2,01/18/2023,07:30 (ET),ATL@DAL,Dallas Mavericks,"Finney-Smith, Dorian",Questionable,Injury/Illness - Right Adductor; Strain
3,01/18/2023,07:30 (ET),ATL@DAL,Dallas Mavericks,"Green, Josh",Questionable,Injury/Illness - Right Elbow; Sprain
4,01/18/2023,07:30 (ET),ATL@DAL,Dallas Mavericks,"Hardaway Jr., Tim",Out,Injury/Illness - Left Ankle; Sprain
...,...,...,...,...,...,...,...
91,01/19/2023,03:00 (ET),CHI@DET,Detroit Pistons,"Diallo, Hamidou",Probable,Injury/Illness - Right Calf; Soreness
92,01/19/2023,03:00 (ET),CHI@DET,Detroit Pistons,"Duren, Jalen",Questionable,Injury/Illness - Right Ankle; Soreness
93,01/19/2023,03:00 (ET),CHI@DET,Detroit Pistons,"Joseph, Cory",Questionable,Injury/Illness - Right Knee; Soreness
94,01/19/2023,03:00 (ET),CHI@DET,Detroit Pistons,"Noel, Nerlens",Questionable,Injury/Illness - Left Foot; Soreness


['Game Date', 'Game Time', 'Matchup', 'Team', 'Player Name', 'Current Status', 'Reason']
['01/18/2023', '07:30 (ET)', 'ATL@DAL', 'Atlanta Hawks', 'Williams, Donovan', 'Out', 'G League - Two-Way']
['', '', 'Dallas Mavericks', 'Doncic, Luka', 'Probable', 'Injury/Illness - Left Ankle; Soreness', '']
['', '', 'Finney-Smith, Dorian', 'Questionable', 'Injury/Illness - Right Adductor; Strain', '', '']
['', '', '', 'Green, Josh', 'Questionable', 'Injury/Illness - Right Elbow; Sprain', '']
['', '', '', 'Hardaway Jr., Tim', 'Out', 'Injury/Illness - Left Ankle; Sprain', '']
['', '', '', 'Kleber, Maxi', 'Out', 'Injury/Illness - Right Hamstring; Tear', '']
['', '', '', 'Wright IV, McKinley', 'Out', 'Injury/Illness - Right Foot; Sprain', '']
['', 'WAS@NYK', 'New York Knicks', 'Jeffries, DaQuan', 'Out', 'G League - Two-Way', '']
['', '', '', 'Keels, Trevor', 'Out', 'G League - Two-Way', '']
['', '', 'Washington Wizards', 'Beal, Bradley', 'Available', 'Injury/Illness - Left Hamstring; Left hamstring s

In [19]:
a = test[0]

In [20]:
print(a)

            Game Date            Game Time               Matchup  \
0          01/18/2023           07:30 (ET)               ATL@DAL   
1                 NaN                  NaN      Dallas Mavericks   
2                 NaN                  NaN  Finney-Smith, Dorian   
3                 NaN                  NaN                   NaN   
4                 NaN                  NaN                   NaN   
5                 NaN                  NaN                   NaN   
6                 NaN                  NaN                   NaN   
7                 NaN              WAS@NYK       New York Knicks   
8                 NaN                  NaN                   NaN   
9                 NaN                  NaN    Washington Wizards   
10                NaN                  NaN                   NaN   
11                NaN                  NaN                   NaN   
12                NaN                  NaN                   NaN   
13         08:00 (ET)              CHA@HOU     C

In [5]:
a = requests.get(link)

In [7]:
read_pdf(a.read())


AttributeError: 'Response' object has no attribute 'read'